In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

import cx_Oracle
import csv

#sql operations
SQL="SELECT * FROM training_data"
connection = cx_Oracle.connect('hr/hr@localhost:1521/xe')
cursor = connection.cursor()
cursor.execute(SQL)
result = cursor.fetchall() 
sqlResult = [list(i) for i in result]

cursor.close()
connection.close()
#DONE with sql

trainArray = np.asarray(sqlResult)



In [2]:
# Split-out validation dataset

x = trainArray[:,0:7]
y = trainArray[:,7]

#random_state can be any number. This is to replicate results over multiple runs
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(x, y, test_size=.2, random_state=7)
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(x_train, y_train)

#find accuracy score
predictions = knn.predict(x_validation)
score = accuracy_score(y_validation, predictions)
print("Accuracy Score = %d" % score)

# Make predictions on test dataset
resultColNames = ['INT_AMB_TEMP','EXT_AMB_TEMP','INPUT_CUR','INPUT_VOLT','VIB_X_AXIS','VIB_Y_AXIS','VIB_Z_AXIS']
testData =  pd.read_csv('C:\BIGDATA\data\sensor.test.data.csv',names=resultColNames) 
testArray = testData.values
unknown = testArray[:,0:7]

predictions = knn.predict(unknown)
# Merge predictions column with Test Data

testData['Result']= predictions

print (testData)



Accuracy Score = 1
   INT_AMB_TEMP  EXT_AMB_TEMP  INPUT_CUR  INPUT_VOLT  VIB_X_AXIS  VIB_Y_AXIS  \
0            41            30         12         115    2.696955   -0.894902   
1            41            30         20         115    2.696955   -0.894902   
2            91            30         12         115    2.696955   -0.894902   
3            41            30         12         135    2.696955   -0.894902   
4            51            45         18         120    2.696955   -0.894902   
5            91            50         18         120    7.455679   -3.678868   
6            85            55         19         125    8.455679   -4.678868   

   VIB_Z_AXIS                    Result  
0   -3.533973        NORMAL_OP_30_DEG_C  
1   -3.533973    OVER_CURRENT_ FAULT_OP  
2   -3.533973        OVER_TEMP_FAULT_OP  
3   -3.533973  INPUT_OVER_VOLT_FAULT_OP  
4   -3.533973      ABNORMAL_OP_30_DEG_C  
5   -9.678345        BEARING_DEGRADE_OP  
6   -8.678345        BEARING_DEGRADE_OP  
